In [2]:
import nltk
import re
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import TweetTokenizer
from nltk.corpus import sentence_polarity
import random
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
from Subjectivity import *
from nltk.stem import PorterStemmer
from nltk.metrics import precision
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.metrics import ConfusionMatrix

In [385]:
nltk.download('averaged_perceptron_tagger')
nltk.download('sentence_polarity')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/komal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentence_polarity to
[nltk_data]     /Users/komal/nltk_data...
[nltk_data]   Package sentence_polarity is already up-to-date!
[nltk_data] Downloading package punkt to /Users/komal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
mycorpus = PlaintextCorpusReader('.','clothing_shoes_jewelry.txt')
type(mycorpus.fileids())

list

In [4]:
file_id = mycorpus.fileids()
file_contents = []
file = file_id[0]
file_handle = open(file, 'r')
file_content = file_handle.read()
file_handle.close()
print(file_content[:500])

reviewerID:A1KLRMWW2FWPL4
asin:0000031887
reviewerName:Amazon Customer "cameramom"
helpful:[0, 0]
reviewText:This is a great tutu and at a really great price. It doesn't look cheap at all. I'm so glad I looked on Amazon and found such an affordable tutu that isn't made poorly. A++
overall:5.0
summary:Great tutu-  not cheaply made
unixReviewTime:1297468800
reviewTime:02 12, 2011

reviewerID:A2G5TCU2WDFZ65
asin:0000031887
reviewerName:Amazon Customer
helpful:[0, 0]
reviewText:I bought this for my 


In [5]:
# extract all reviews

pword = re.compile('reviewerID:(.*)\nasin:(.*)\nreviewerName:(.*)\nhelpful:(.*)\nreviewText:(.*)\n(overall:.*)\nsummary:(.*)\nunixReviewTime:(.*)\nreviewTime:(.*)\n')
reviews = re.findall(pword,file_content)

In [6]:
len(reviews)

278225

In [7]:
# extract the year
int(reviews[11][8][7:])

2014

In [9]:
# count number of reviews in 2014
counter = 0
for i in range(len(reviews)):
    year = int(reviews[i][8][7:])
    if (year == 2014):
        # only [i][4] corresponds to the reviewText field. 
        counter+=1

print("number of reviews in 2014 : " + str(counter))

number of reviews in 2014 : 67614


In [69]:
# reviews_2014 only first 5500 contains the reviews of year 2014
reviews_2014 = []
counter = 0
for i in range(len(reviews)):
    year = int(reviews[i][8][7:])
    if (year == 2014):
        # only [i][4] corresponds to the reviewText field. 
        reviews_2014.append(reviews[i][4])
        counter+=1
        if (counter >= 5500):
            break


In [70]:
len(reviews_2014)

5500

In [71]:
reviews_2014[:2]

['We bought several tutus at once, and they are got high reviews. Sturdy and seemingly well-made. The girls have been wearing them regularly, including out to play, and the tutus have stood up well. Fits the 3-yr old & the 5-yr old well. Clearly plenty of room to grow. Only con is that when the kids pull off the tutus, the waste band gets twisted, and an adult has to un-tangle. But this is not difficult.',
 "Thank you Halo Heaven great product for Little Girls.  My Great Grand Daughters Love these Tutu's.  Will buy more from this seller.  Made well and cute on the girls.  Thanks for a great product.NEVER BUY FROM DRESS UP DREAMS........I will buy more as long as I don't buy from &#34;Dress Up Dreams&#34;  I never rec'd or order in FL. Only rec'd pink, the purple one was missing.  Company is a rip off.  REFUSES to make good on purchase...... Real creeps."]

In [409]:
# Sentence level tokenization
sentences=[nltk.sent_tokenize(sent) for sent in reviews_2014]

In [411]:
sentences[-2]

["It won't hold heavy canned goods like a cotton canvas bag, and is a little smaller than paper grocery bag.",
 'Good for the beach or knitting carryall.',
 'The fabric seems to be polyester with a waterproof coating inside that also stabilizes it.',
 'The plastic smell does go away in a couple of days IF you open them up to air out.',
 "There's no stiffener in the bottom so it folds very flat.",
 'While the seams are stitched adequately, they are not finished on the inside, though a couple of colors do have taped seams.',
 'But for $3.50, they are a bargain.']

In [431]:
# tokenize each review
# Sentence level tokenization, I am using Tweet Tokenizer because 
# if we use nltk.sent_tokenize, the words like didn't gets split, which we dont want.

tknzr = TweetTokenizer()
original_tokenized_review_sentences = []
for each_review in sentences:
    # tokenize each review into words by splitting them into different sentences
    sent_wordlevel=[tknzr.tokenize(sent) for sent in each_review]
    for each in sent_wordlevel:
        original_tokenized_review_sentences.append(each)
        
print(len(original_tokenized_review_sentences))
print(original_tokenized_review_sentences[-3:-1])

21629
[['I', 'took', 'it', 'to', 'ups', 'myself', 'and', 'paid', 'the', 'return', 'with', '$', '100.00', 'insurance', 'secured', 'for', 'what', 'look', 'like', 'are', 'boots', 'worth', '$', '20', '.'], ["I'm", 'sorry', 'I', 'have', 'not', 'had', 'any', 'issues', 'ever', 'with', 'Amazon', 'cause', 'your', 'stuff', 'is', 'great', 'and', "I'm", 'always', 'happy', 'with', 'my', 'purchases', 'but', 'since', 'I', 'kept', 'my', 'receipts', 'on', 'the', 'returns', 'of', 'these', 'items', 'please', 'ask', 'your', '3rd', 'party', 'seller', 'what', 'happen', '!']]


In [436]:
# converting the sentences to lower case to have the uniformity during classification
tokenized_review_sentences = []
# sentences now has the originally selected sentences from the reiviews file
# and lower_case_sentences has the 
for sentence in original_tokenized_review_sentences:
    tokenized_review_sentences.append([item.lower() for item in sentence])
    
print(tokenized_review_sentences[-3:-1])

[['i', 'took', 'it', 'to', 'ups', 'myself', 'and', 'paid', 'the', 'return', 'with', '$', '100.00', 'insurance', 'secured', 'for', 'what', 'look', 'like', 'are', 'boots', 'worth', '$', '20', '.'], ["i'm", 'sorry', 'i', 'have', 'not', 'had', 'any', 'issues', 'ever', 'with', 'amazon', 'cause', 'your', 'stuff', 'is', 'great', 'and', "i'm", 'always', 'happy', 'with', 'my', 'purchases', 'but', 'since', 'i', 'kept', 'my', 'receipts', 'on', 'the', 'returns', 'of', 'these', 'items', 'please', 'ask', 'your', '3rd', 'party', 'seller', 'what', 'happen', '!']]


In [437]:
print(len(tokenized_review_sentences))

21629


In [209]:
# Stop Words
stop_words = set(stopwords.words('english'))
print(stop_words)

negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

neg_stop_words = []
print(type(stop_words))
for word in stop_words:
    if (word in negationwords) or (word.endswith("n't")):
        neg_stop_words.append(word)
    
#print(neg_stop_words)
#print(stop_words)      
neg_stop_words = set(neg_stop_words)
new_stop_words = []
new_stop_words = list(stop_words - neg_stop_words)


{'each', 'above', 'again', 'but', 'mightn', 'then', 'in', 'herself', "it's", 'about', 'o', "you're", 'did', 'shouldn', 'himself', 'him', 'the', 'both', 'myself', 'at', "you'd", 'now', 'couldn', 'against', 'mustn', 'they', 't', 'over', 'no', 'more', 'to', "she's", 'there', "shouldn't", 've', 'during', 'through', "that'll", 'y', 'do', 'll', 'i', 'his', 'hers', 'that', "didn't", 'with', "wasn't", 'any', 'didn', 'shan', 'why', 'd', 'ours', 'having', 'an', "weren't", 'own', 'its', 'while', 'itself', 'until', "isn't", 'your', 'some', 'such', 'have', 'will', 'from', 'what', 'were', 'or', 'm', 'are', 'he', 'down', "mustn't", 'which', "haven't", 'yourself', 'between', 'all', 'it', 'should', "shan't", 'am', "couldn't", 's', 'few', 'be', 'haven', 'yourselves', 'ain', 'whom', 'was', 'before', "mightn't", 'these', "should've", 'been', 'once', 'wasn', "wouldn't", 'aren', 'so', 'only', 'and', 'you', 'nor', 'too', 'our', 'other', "hadn't", 'doesn', 'here', 'ma', 'isn', 'when', 'can', 'wouldn', 'does',

In [ ]:
# Few stop words I thought will affect the classification, 
# like too, again 
# but as we can see in the below results, it really doesnt affect 
# the classification

print(new_stop_words)

In [ ]:
list(swn.senti_synsets("too"))

In [211]:
breakdown3 = swn.senti_synset('besides.r.02')
print(breakdown3.pos_score())
print(breakdown3.neg_score())
print(breakdown3.obj_score())

0.0
0.0
1.0


In [212]:
list(swn.senti_synsets("again"))

[SentiSynset('again.r.01')]

In [213]:
breakdown3 = swn.senti_synset('again.r.01')
print(breakdown3.pos_score())
print(breakdown3.neg_score())
print(breakdown3.obj_score())

0.0
0.0
1.0


In [214]:
list(swn.senti_synsets("very"))

[SentiSynset('very.s.01'),
 SentiSynset('identical.s.02'),
 SentiSynset('very.r.01'),
 SentiSynset('very.r.02')]

In [215]:
breakdown3 = swn.senti_synset('very.s.01')
print(breakdown3.pos_score())
print(breakdown3.neg_score())
print(breakdown3.obj_score())

0.5
0.0
0.5


## Sentiment Analysis

### Sentiment Classification - Bag of words approach

In [118]:
sentences = sentence_polarity.sents()
print(sentence_polarity.categories())
documents = [(sent, cat) for cat in sentence_polarity.categories() 
    for sent in sentence_polarity.sents(categories=cat)]

In [119]:
documents = [(sent, cat) for cat in sentence_polarity.categories() 
    for sent in sentence_polarity.sents(categories=cat)]

In [120]:
random.shuffle(documents)

In [121]:
all_words_list = [word for (sent,cat) in documents for word in sent]
all_words = nltk.FreqDist(all_words_list)
word_items = all_words.most_common(2000)
word_features = [word for (word, freq) in word_items]

In [122]:
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [123]:
featuresets = [(document_features(d,word_features), c) for (d,c) in documents]

In [124]:
train_set, test_set = featuresets[1000:], featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print (nltk.classify.accuracy(classifier, test_set))

0.755


In [216]:
# Trying with 3000 most frequent bag of words
sentences = sentence_polarity.sents()
print(sentence_polarity.categories())
documents = [(sent, cat) for cat in sentence_polarity.categories() 
    for sent in sentence_polarity.sents(categories=cat)]

random.shuffle(documents)

all_words_list = [word for (sent,cat) in documents for word in sent]
all_words = nltk.FreqDist(all_words_list)
word_items = all_words.most_common(3000)
word_features = [word for (word, freq) in word_items]

def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(d,word_features), c) for (d,c) in documents]

train_set, test_set = featuresets[1000:], featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print (nltk.classify.accuracy(classifier, test_set))

['neg', 'pos']
0.766


In [217]:
# recalculating Document Feature after removing stop words

sentences = sentence_polarity.sents()
print(sentence_polarity.categories())
documents = [(sent, cat) for cat in sentence_polarity.categories() 
    for sent in sentence_polarity.sents(categories=cat)]

random.shuffle(documents)

# all_word_list after removing the stop words
all_words_list = [word for (sent,cat) in documents for word in sent if word not in new_stop_words]
all_words = nltk.FreqDist(all_words_list)
word_items = all_words.most_common(3000)
word_features = [word for (word, freq) in word_items]

# bag of words approach
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

# define the feature sets using the document_features
featuresets = [(document_features(d,word_features), c) for (d,c) in documents]

# Train and test your model for accuracy
train_set, test_set = featuresets[1000:], featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print (nltk.classify.accuracy(classifier, test_set))



['neg', 'pos']
0.767


### Sentiment Classification - Subjectivity Count features

In [440]:
SLpath = 'subjclueslen1-HLTEMNLP05.tff'
SL = readSubjectivity(SLpath)
print(SL['absolute'])

# define the features, to find out the feature_set
def SL_features(document, word_features, SL):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
        # count variables for the 4 classes of subjectivity
        weakPos = 0
        strongPos = 0
        weakNeg = 0
        strongNeg = 0
        for word in document_words:
            if word in SL:
                strength, posTag, isStemmed, polarity = SL[word]
                if strength == 'weaksubj' and polarity == 'positive':
                    weakPos += 1
                if strength == 'strongsubj' and polarity == 'positive':
                    strongPos += 1
                if strength == 'weaksubj' and polarity == 'negative':
                    weakNeg += 1
                if strength == 'strongsubj' and polarity == 'negative':
                    strongNeg += 1
                features['positivecount'] = weakPos + (2 * strongPos)
                features['negativecount'] = weakNeg + (2 * strongNeg)      
    return features


#define the feature set for performinh the classification
# word features here is the revised word features after removing the stop words
SL_featuresets = [(SL_features(d, word_features, SL), c) for (d,c) in documents]

print(SL_featuresets[0][0]['positivecount'])
print(SL_featuresets[0][0]['negativecount'])

train_set, test_set = SL_featuresets[1000:], SL_featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

['strongsubj', 'adj', False, 'neutral']
2
0
0.73


### Sentiment Analysis - Negation Features

In [222]:
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = False
        features['contains(NOT{})'.format(word)] = False
    # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['contains(NOT{})'.format(document[i])] = (document[i] in word_features)
        else:
            features['contains({})'.format(word)] = (word in word_features)
    return features

In [223]:
# this word_features is the list of word_features after removing the stop words
NOT_featuresets = [(NOT_features(d, word_features, negationwords), c) for (d, c) in documents]
NOT_featuresets[0][0]['contains(NOTlike)']
NOT_featuresets[0][0]['contains(always)']

train_set, test_set = NOT_featuresets[1000:], NOT_featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

classifier.show_most_informative_features(30)

0.783
Most Informative Features
          contains(warm) = True              pos : neg    =     19.8 : 1.0
    contains(engrossing) = True              pos : neg    =     19.1 : 1.0
       contains(generic) = True              neg : pos    =     16.3 : 1.0
       contains(routine) = True              neg : pos    =     14.9 : 1.0
          contains(flat) = True              neg : pos    =     14.5 : 1.0
      contains(mediocre) = True              neg : pos    =     14.3 : 1.0
      contains(supposed) = True              neg : pos    =     14.3 : 1.0
        contains(boring) = True              neg : pos    =     13.8 : 1.0
     contains(inventive) = True              pos : neg    =     13.1 : 1.0
      contains(powerful) = True              pos : neg    =     12.5 : 1.0
   contains(mesmerizing) = True              pos : neg    =     11.7 : 1.0
     contains(absorbing) = True              pos : neg    =     11.7 : 1.0
    contains(refreshing) = True              pos : neg    =     11.7

In [245]:
def make_sentence(tokenized_review_sentence):
    sentence = ""
    for i in range(len(tokenized_review_sentence)):
        sentence+=  tokenized_review_sentence[i] + " "
    return sentence

In [441]:
positive_file_handle = open("positive.txt","w")
negative_file_handle = open("negative.txt","w")

for i in range(len(tokenized_review_sentences)):
    sent, orig_sent = tokenized_review_sentences[i], original_tokenized_review_sentences[i]
    if((classifier.classify(document_features(sent,word_features))) == 'pos'):
        positive_file_handle.write(make_sentence(orig_sent))
        positive_file_handle.write("\n")
    else:
        negative_file_handle.write(make_sentence(orig_sent))
        negative_file_handle.write("\n")

positive_file_handle.close()
negative_file_handle.close()

In [344]:
# Defining classifier, based on only adjectives and exclaimatory marks

sentences = sentence_polarity.sents()
print(sentence_polarity.categories())
documents = [(sent, cat) for cat in sentence_polarity.categories() 
    for sent in sentence_polarity.sents(categories=cat)]

random.shuffle(documents)

# all_word_list after removing the stop words
all_words_list = [word for (sent,cat) in documents for word in sent if word not in new_stop_words]

all_words = nltk.FreqDist(all_words_list)
word_items = all_words.most_common(15000)

# we can see that all_words_list currently has all the words like nouns
# adjectives, pronouns, exclaimation marks, proper nouns, etc
# print(word_items)

# Since we know that classification of sentiment is purely based on the adjectives
# negative words, etc

# lets remove all the special symbols from the list

print(type(word_items))
word_dict = dict(word_items)
print(type(word_dict))

# remove all non - alpha characters
pattern = "[^A-Za-z]+"
prog = re.compile(pattern)
refined_word_dict = dict()
for key in word_dict:
    if (prog.match(key)):
        # .,"--)?(:';!-2*[a]902002 all such words are removed from the most frequent words
        if (len(key)<=5):
            print("removing tuple with key "+ key)
        else:
            value = word_dict[key]
        refined_word_dict[key] = value

    else:
        value = word_dict[key]
        refined_word_dict[key] = value

# adding 2 important words externally to the word_dict
refined_word_dict['love'] = 100
refined_word_dict['hate'] = 100
        
print("Len After removing special symbols "+str(len(refined_word_dict)))
word_dict = dict()

# removing nouns, determinants from the list

for key in refined_word_dict:
    word_tokens = []
    word_tokens.append(key)
    tag = nltk.pos_tag(word_tokens) 
    tag = tag[0][1]
    if(tag == 'NN' or tag == 'NNS'):
        # print("removing Noun tuple with key "+ key)
        continue
    #elif (tag == 'VB' or tag == 'VBD' or tag == 'VBN' or tag == 'VBP' or tag == 'VBZ' ):
        # print("removing Verb tuple with key " + key)
    #    continue
    else:
        value = refined_word_dict[key]
        word_dict[key] = value
        # print(tag)
        
print("Len after removing nouns and verbs " + str(len(word_dict)))
# print(word_dict)
import copy

word_dict_copy = copy.copy(word_dict)

# stemming
ps = PorterStemmer()
for key in word_dict_copy:
    new_key = key
    new_key = ps.stem(new_key)
    new_value = word_dict_copy[key]
    word_dict[new_key] = new_value
    
# converting dictionary to list again
word_items = word_dict.items()
word_features = [word for (word, freq) in word_items]



# print(word_features)



['neg', 'pos']
<class 'list'>
<class 'dict'>
removing tuple with key .
removing tuple with key ,
removing tuple with key "
removing tuple with key --
removing tuple with key )
removing tuple with key ?
removing tuple with key (
removing tuple with key :
removing tuple with key '
removing tuple with key ;
removing tuple with key !
removing tuple with key -
removing tuple with key 
removing tuple with key 2
removing tuple with key *
removing tuple with key [a]
removing tuple with key 90
removing tuple with key 2002
removing tuple with key 10
removing tuple with key 
removing tuple with key 'the
removing tuple with key &
removing tuple with key 20
removing tuple with key [the
removing tuple with key '70s
removing tuple with key '60s
removing tuple with key 51
removing tuple with key 15
removing tuple with key /
removing tuple with key 80
removing tuple with key 4ever
removing tuple with key 1
removing tuple with key 101
removing tuple with key 8
removing tuple with key 30
removing tuple

In [345]:
SLpath = 'subjclueslen1-HLTEMNLP05.tff'
SL = readSubjectivity(SLpath)
print(SL['absolute'])

# define the features, to find out the feature_set
def SL_features(document, word_features, SL):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
        # count variables for the 4 classes of subjectivity
        weakPos = 0
        strongPos = 0
        weakNeg = 0
        strongNeg = 0
        for word in document_words:
            if word in SL:
                strength, posTag, isStemmed, polarity = SL[word]
                if strength == 'weaksubj' and polarity == 'positive':
                    weakPos += 1
                if strength == 'strongsubj' and polarity == 'positive':
                    strongPos += 1
                if strength == 'weaksubj' and polarity == 'negative':
                    weakNeg += 1
                if strength == 'strongsubj' and polarity == 'negative':
                    strongNeg += 1
                features['positivecount'] = weakPos + (2 * strongPos)
                features['negativecount'] = weakNeg + (2 * strongNeg)      
    return features


#define the feature set for performinh the classification
# word features here is the revised word features after removing the stop words
SL_featuresets = [(SL_features(d, word_features, SL), c) for (d,c) in documents]

print(SL_featuresets[0][0]['positivecount'])
print(SL_featuresets[0][0]['negativecount'])

train_set, test_set = SL_featuresets[1000:], SL_featuresets[:1000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

['strongsubj', 'adj', False, 'neutral']
2
0
0.73


In [367]:
reflist = []
testlist = []
for (features, label) in test_set:
    reflist.append(label) 
    testlist.append(classifier.classify(features))

In [371]:
print(reflist[:30] )
print(testlist[:30])

['pos', 'pos', 'neg', 'pos', 'pos', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'pos', 'neg', 'neg', 'neg', 'neg', 'pos', 'neg', 'pos', 'neg', 'neg', 'neg', 'pos', 'pos']
['pos', 'neg', 'neg', 'pos', 'pos', 'neg', 'pos', 'neg', 'neg', 'pos', 'pos', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'neg', 'pos', 'pos', 'neg', 'pos', 'neg', 'pos', 'neg', 'neg', 'pos', 'neg', 'pos']


In [392]:
dir(nltk.metrics)
from nltk.metrics import *

In [402]:
reflist = []
testlist = []
for (features, label) in test_set:
    reflist.append(label) 
    testlist.append(classifier.classify(features))

In [407]:
cm = ConfusionMatrix(reflist, testlist)
print(cm)

    |   n   p |
    |   e   o |
    |   g   s |
----+---------+
neg |<358>146 |
pos | 125<371>|
----+---------+
(row = reference; col = test)



In [442]:
from sklearn.svm import LinearSVC
from nltk.classify.scikitlearn import SklearnClassifier
classifier=nltk.classify.SklearnClassifier(LinearSVC()).train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.729

In [372]:
# calculating true negative, false positive, false negative, true positive
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(reflist, testlist).ravel()
(tn, fp, fn, tp)

(353, 151, 119, 377)